# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
pip install gmaps

     |████████████████████████████████| 1.1 MB 5.0 MB/s eta 0:00:01


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076111 sha256=5968f49a98695935389c9dcc352de832c7d45c4380762f8dd12ab1462b531286
  Stored in directory: /Users/miriam/Library/Caches/pip/wheels/66/ab/29/3032938612273665f0fa4d2fab4e3ecfe86480eaaa40aaee91
Successfully built gmaps
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_csv_file = "../WeatherPy/output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ust-kuyga,70.0167,135.6000,20.84,94.0,100.0,9.75,RU,1.665267e+09
1,punta arenas,-53.1500,-70.9167,46.51,66.0,75.0,19.57,CL,1.665266e+09
2,hermanus,-34.4187,19.2345,53.80,85.0,77.0,8.59,ZA,1.665266e+09
3,okakarara,-20.5833,17.4333,68.41,14.0,0.0,5.77,NaN,1.665267e+09
4,rudbar,30.1500,62.6000,61.61,15.0,0.0,7.99,AF,1.665267e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,marsaxlokk,35.8419,14.5431,73.08,77.0,0.0,2.30,MT,1.665267e+09
220,ampanihy,-24.7000,44.7500,76.28,40.0,0.0,3.29,MG,1.665267e+09
320,sarai alamgir,32.9012,73.7559,71.40,66.0,0.0,2.86,PK,1.665267e+09
322,buraydah,26.3260,43.9750,77.68,12.0,0.0,5.82,SA,1.665267e+09
358,rome,34.2570,-85.1647,70.47,32.0,0.0,3.00,US,1.665267e+09
402,surt,31.2089,16.5887,73.85,59.0,0.0,8.25,LY,1.665267e+09
437,nkhata bay,-11.6066,34.2907,72.99,49.0,0.0,5.70,MW,1.665267e+09
536,southaven,34.9890,-90.0126,70.92,31.0,0.0,9.22,US,1.665267e+09
557,astoria,46.1879,-123.8313,73.47,79.0,0.0,8.05,US,1.665267e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 5: marsaxlokk.
Closest hotel is Water's Edge.
------------
Retrieving Results for Index 220: ampanihy.
Closest hotel is HOTEL SHARRON AMPANIHY OUEST.
------------
Retrieving Results for Index 320: sarai alamgir.
Closest hotel is Tulip Riverside Hotel And Restaurant Jehlum.
------------
Retrieving Results for Index 322: buraydah.
Closest hotel is Mövenpick Qassim.
------------
Retrieving Results for Index 358: rome.
Closest hotel is La Quinta Inn & Suites by Wyndham Rome.
------------
Retrieving Results for Index 402: surt.
Closest hotel is فندق المدينة.
------------
Retrieving Results for Index 437: nkhata bay.
Closest hotel is Mayoka Village.
------------
Retrieving Results for Index 536: southaven.
Closest hotel is Hilton Garden Inn Memphis/Southaven, MS.
------------
Retrieving Results for Index 557: astoria.
Closest hotel is Best Western Astoria Bayfront Hotel.
------------


In [7]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,marsaxlokk,35.8419,14.5431,73.08,77.0,0.0,2.30,MT,1.665267e+09,Water's Edge
220,ampanihy,-24.7000,44.7500,76.28,40.0,0.0,3.29,MG,1.665267e+09,HOTEL SHARRON AMPANIHY OUEST
320,sarai alamgir,32.9012,73.7559,71.40,66.0,0.0,2.86,PK,1.665267e+09,Tulip Riverside Hotel And Restaurant Jehlum
322,buraydah,26.3260,43.9750,77.68,12.0,0.0,5.82,SA,1.665267e+09,Mövenpick Qassim
358,rome,34.2570,-85.1647,70.47,32.0,0.0,3.00,US,1.665267e+09,La Quinta Inn & Suites by Wyndham Rome
402,surt,31.2089,16.5887,73.85,59.0,0.0,8.25,LY,1.665267e+09,فندق المدينة
437,nkhata bay,-11.6066,34.2907,72.99,49.0,0.0,5.70,MW,1.665267e+09,Mayoka Village
536,southaven,34.9890,-90.0126,70.92,31.0,0.0,9.22,US,1.665267e+09,"Hilton Garden Inn Memphis/Southaven, MS"
557,astoria,46.1879,-123.8313,73.47,79.0,0.0,8.05,US,1.665267e+09,Best Western Astoria Bayfront Hotel


In [10]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
# Add the layer to the map
fig.add_layer(hotel_layer)
#display map
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
fig

Figure(layout=FigureLayout(height='420px'))